In [2]:
#USER INPUT
# Choose the three one-vs-all models you want to use
classes = ['Glitch', 'NS', 'BBH']
model_names = ['Glitch_vs_all_model', 'NS_vs_all_model', 'BBH_vs_all_model'] # Please be sure classes and model_names follow the same ordering

#Thresholds used for each classifier. 
glitchthrsh = 32.33
NSthrsh = 62.16
BBHthrsh = 31.18

# Import Libraries

In [3]:
import requests
import numpy as np
import tensorflow as tf
from ligo.skymap import io, distance
from astropy.io import fits
from reproject import reproject_from_healpix
from tensorflow import keras

  Using cached matplotlib-3.4.3-cp37-cp37m-manylinux1_x86_64.whl (10.3 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.3
    Uninstalling matplotlib-3.1.3:
      Successfully uninstalled matplotlib-3.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


  Using cached matplotlib-3.1.3-cp37-cp37m-manylinux1_x86_64.whl (13.1 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.4.3
    Uninstalling matplotlib-3.4.3:
      Successfully uninstalled matplotlib-3.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ligo-skymap 0.5.3 requires matplotlib>=3.4.0, but you have matplotlib 3.1.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


# Load Models

In [4]:
# Load each model and their weight's
i=0
loaded_models = []
loaded_weights = []
for i, name in enumerate(model_names):
  with open(name + '.json', 'r') as json_file:
    loaded_models.append(keras.models.model_from_json(json_file.read()))
    loaded_weights.append(loaded_models[i].load_weights(name + '.h5'))

# Load GraceDB Data

In [5]:
target_header = fits.Header.fromstring("""
NAXIS   =                    2
NAXIS1  =                  360
NAXIS2  =                  180
CTYPE1  = 'RA---CAR'
CRPIX1  =                180.5
CRVAL1  =                180.0
CDELT1  =                   -1
CUNIT1  = 'deg     '
CTYPE2  = 'DEC--CAR'
CRPIX2  =                 90.5
CRVAL2  =                  0.0
CDELT2  =                    1
CUNIT2  = 'deg     '
COORDSYS= 'icrs    '
""", sep='\n')

In [6]:
# Normalization factors (from the training) to normalize distances and Bayes factors
training_norms = {'mean_distance': 10319.911, 'max_distance': 14877.898000000001,
                  'skymap': 0.005, 'vol0': 240.611,
                  'vol1': 959.2710000000001, 'vol2': 68.922,
                  'logBCI': 38.364000000000004, 'logBSN': 7954.571}

In [7]:
def nan_invalid(data, invalid_value):
    """Turn invalid values into numpy.nan"""
    invalid_indices = numpy.where(data==invalid_value)
    for idx in invalid_indices:
        data[idx] = numpy.nan
    return data

def prepare_data(fits_file):

    (prob, mu, sigma, norm), metadata = io.read_sky_map(fits_file, distances=True, nest=None)

    meandist, std = metadata['distmean'], metadata['diststd']
    maxdist = meandist + 2.5 * std
    # distances and Bayes factors must be normalized by maximum in the training set
    mean_distance = meandist / training_norms['mean_distance']
    max_distance = maxdist / training_norms['max_distance']
    logBCI = metadata['log_bci'] / training_norms['logBCI']
    logBSN = metadata['log_bsn'] / training_norms['logBSN']
    
    network = metadata['instruments']
    dets = []
    for ifo in ['H1', 'L1', 'V1']:
        dets.append(1) if ifo in network else dets.append(0)

    img_data, norm_img, norms = dict(), dict(), dict()

    # Reproject skymap data to rectangle
    with np.errstate(invalid='ignore'):
        img, mask = reproject_from_healpix((prob, 'ICRS'), target_header,
                                    nested=metadata['nest'], hdu_in=None,
                                    order='bilinear', field=0)
        img_data['skymap'] = img
        
    # Calculate volume projections    
    rot = np.ascontiguousarray(distance.principal_axes(prob, mu, sigma))
    dpi = 150
    figure_width = 3.5
    imgwidth = int(dpi * figure_width / 2)
    s = np.linspace(-maxdist, maxdist, imgwidth)
    xx, yy = np.meshgrid(s, s)
    for iface, (axis0, axis1) in enumerate(((1,0), (0,2), (1,2))):
        density = distance.volume_render(xx.ravel(), yy.ravel(), maxdist,
                                        axis0, axis1, rot, False, prob, mu,
                                        sigma, norm).reshape(xx.shape)
        img_data['vol{}'.format(iface)] = density
    
    for column in img_data.keys():
        # Normalize img data
        norm = np.max(img_data[column])
        img = img_data[column] / norm

        # Downsize img data using maxpooling
        x = np.reshape(img, (1, len(img), len(img[0]), 1))
        # To avoid tensorflow warnings
        x = tf.cast(x, tf.float32)
        maxpool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))
        norm_img[column] = np.array(maxpool(x))
        # Normalize norms
        norms[column] = norm / training_norms[column]

    # Stack volume images
    dist_columns = ['vol0', 'vol1', 'vol2']
    # img_data has shape (1, 131, 131, 1), we need to reshape to (1, 131, 131) for stacking
    stacked_volume = np.stack([np.reshape(norm_img[column], (1, 131, 131)) for column in dist_columns], axis=-1)
    stacked_volnorms = np.stack([norms[column] for column in dist_columns], axis=-1)
    # Stack distances
    distances = np.stack((mean_distance, max_distance), axis=-1)

    return [stacked_volume, norm_img['skymap'], np.reshape(dets, (1,3)), np.reshape(distances, (1,2)),
            np.reshape(norms['skymap'], (1,1)), np.reshape(stacked_volnorms, (1,3)), np.reshape(logBSN, (1,1)), np.reshape(logBCI, (1,1))]

def predict(loaded_models, data):
    """Use loaded model to predict result
    
    Keyword arguments:
    loaded_models: machine-learning models to use for predictions
    data: pre-processed data from FITS file
    """
    glitchscore = 100*tf.squeeze(loaded_models[0](data), [-1]).numpy()
    NSscore = 100*tf.squeeze(loaded_models[1](data), [-1]).numpy()
    BBHscore = 100*tf.squeeze(loaded_models[2](data), [-1]).numpy()
    
    predictions=[glitchscore, NSscore, BBHscore]

    if glitchscore >= glitchthrsh:
        hier_pred='Glitch'

    elif NSscore >= NSthrsh:
        hier_pred='NS'

    elif BBHscore >= BBHthrsh:
        hier_pred='BBH'

    else: 
        m= max((glitchscore - glitchthrsh),  (NSscore- NSthrsh), (BBHscore- BBHthrsh))
        if m == (glitchscore - glitchthrsh):
          hier_pred='Glitch'
        elif m == (NSscore- NSthrsh):
          hier_pred='NS'
        elif m == (BBHscore- BBHthrsh):
          hier_pred='BBH'
      
    print(event_name)

    for i in range(len(loaded_models)):
      print(classes[i] + " Confidence Score: "+ str(np.round(predictions[i],1)) + '%')
    print('Hierarchical Prediction: '+ hier_pred)




In [8]:
# Choose candidate from GraceDB and download corresponding FITS file
# See https://gracedb.ligo.org/superevents/public/O3/ for a list of candidates
event_name = 'S200316bj'
event_url = 'https://gracedb.ligo.org/apiweb/superevents/{}/files/'.format(event_name)
r = requests.head(event_url + 'bayestar.multiorder.fits')
try:
    r.headers['Content-Disposition']
    fits_url = event_url + 'bayestar.multiorder.fits'
except KeyError:
    # Older events do not have bayestar.multiorder.fits file
    fits_url = event_url + 'bayestar.fits'
fits_name = '{}.fits'.format(event_name)
!curl --output $fits_name $fits_url

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  759k  100  759k    0     0   500k      0  0:00:01  0:00:01 --:--:--  500k


In [9]:
data = prepare_data(fits_name)

predict(loaded_models, data)

S200316bj
Glitch Confidence Score: [0.2]%
NS Confidence Score: [0.3]%
BBH Confidence Score: [99.3]%
Hierarchical Prediction: BBH
